In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
df_2000 = pd.read_csv('../data_csv/clean_2000_df.csv')
df_2000.head()

,Unnamed: 0,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,0,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2000,purchase-only,1,149.78,150.22
1,1,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2000,purchase-only,2,149.29,149.33
2,2,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2000,purchase-only,3,151.34,150.79
3,3,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2000,purchase-only,4,152.36,151.47
4,4,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2000,purchase-only,5,153.57,151.82


In [2]:
# Drop the non-beneficial ID columns
df_2000 = df_2000.drop(columns = ['Unnamed: 0'], axis= 1)
df_2000.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2000,purchase-only,1,149.78,150.22
1,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2000,purchase-only,2,149.29,149.33
2,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2000,purchase-only,3,151.34,150.79
3,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2000,purchase-only,4,152.36,151.47
4,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2000,purchase-only,5,153.57,151.82


In [3]:
#check for bucketing
# Generate our categorical variable list
cat_2000 = df_2000.dtypes[df_2000.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
df_2000[cat_2000].nunique()

hpi_type        3
hpi_flavor      2
frequency       4
level         162
place_name    162
yr              2
dtype: int64

In [4]:
#11 uniques values needs to be checked 
# Check the unique value counts to see if binning is required
df_2000.level.value_counts()

East North Central Division    20
Pacific Division               20
United States                  20
West North Central Division    20
South Atlantic Division        20
                               ..
Fresno, CA                      4
Gary, IN (MSAD)                 4
Grand Rapids-Kentwood, MI       4
Greensboro-High Point, NC       4
Puerto Rico                     4
Name: level, Length: 162, dtype: int64

In [5]:
#there are a substantial number of datapoints. leave it alone no need to bucket
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_2000[cat_2000]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat_2000)
encode_df.head()

,hpi_type_developmental,hpi_type_distress-free,hpi_type_traditional,hpi_flavor_monthly,hpi_flavor_quarterly,frequency_MSA,frequency_Puerto Rico,frequency_State,frequency_USA or Census Division,"level_Akron, OH",...,place_name_USA,place_name_UT,place_name_VA,place_name_VT,place_name_WA,place_name_WI,place_name_WV,place_name_WY,yr_expanded-data,yr_purchase-only
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
# Merge one-hot encoded features and drop the originals
df_2000 = df_2000.merge(encode_df,left_index=True, right_index=True)
df_2000 = df_2000.drop(cat_2000,1)
df_2000.head()

/Users/joseperez/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,place_id,period,index_nsa,index_sa,hpi_type_developmental,hpi_type_distress-free,hpi_type_traditional,hpi_flavor_monthly,hpi_flavor_quarterly,frequency_MSA,...,place_name_USA,place_name_UT,place_name_VA,place_name_VT,place_name_WA,place_name_WI,place_name_WV,place_name_WY,yr_expanded-data,yr_purchase-only
0,2000,1,149.78,150.22,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2000,2,149.29,149.33,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2000,3,151.34,150.79,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2000,4,152.36,151.47,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2000,5,153.57,151.82,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
# Remove loan status target from features data
y = df_2000.hpi_type_traditional
X = df_2000.drop(columns=["index_sa","index_nsa"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 1.000


In [9]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-05-22 17:23:09.577140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
30/30 [==============================] - 2s 3ms/step - loss: 0.7487 - accuracy: 0.5651
Epoch 2/50
30/30 [==============================] - 0s 2ms/step - loss: 0.2322 - accuracy: 0.9481
Epoch 3/50
30/30 [==============================] - 0s 2ms/step - loss: 0.1070 - accuracy: 0.9884
Epoch 4/50
30/30 [==============================] - 0s 2ms/step - loss: 0.0592 - accuracy: 0.9958
Epoch 5/50
30/30 [==============================] - 0s 2ms/step - loss: 0.0352 - accuracy: 1.0000
Epoch 6/50
30/30 [==============================] - 0s 2ms/step - loss: 0.0219 - accuracy: 1.0000
Epoch 7/50
30/30 [==============================] - 0s 2ms/step - loss: 0.0146 - accuracy: 1.0000
Epoch 8/50
30/30 [==============================] - 0s 2ms/step - loss: 0.0102 - accuracy: 1.0000
Epoch 9/50
30/30 [==============================] - 0s 2ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 10/50
30/30 [==============================] - 0s 2ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 11/50
30/30 [